In [1]:
import ROOT
import numpy as np
from array import array
import multiprocessing as mp
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
N_tr = 10
Ne = 1_000_000/N_tr
M_PI0 = 0.134977
me = 0.000511
alpha = 1.0 / 137.036
PI = np.pi
colors = [ROOT.kBlack, ROOT.kBlue, ROOT.kRed, ROOT.kGreen + 2, ROOT.kOrange + 2, ROOT.kMagenta]
Ne = int(Ne)

In [3]:

def KrollWada(mll, mp, ml, mo=0):
    c1 = 2. * alpha / (3. * PI)
    q = (mll / mp)**2
    eps = (ml / mp)**2
    delta = (mo / mp)**2
    f = (1. + q / (1. - delta))**2 - 4. * q / (1. - delta)**2
    return 2. * c1 / mll * f**1.5 * np.sqrt(1. - 4. * eps / q) * (1. + 2. * eps / q)

def EMTFormFactor(m, b):
    return (1. / (1. - b * m**2))**2

def pi0DalitzPDF(mll):
    return KrollWada(mll, M_PI0, me) * EMTFormFactor(mll, 5.5)

def HagedornYield(pt, mass, A, a, b, p0, n):
    mt = np.sqrt(pt**2 + mass**2 - M_PI0**2)
    return pt * A / (2. * PI) * (np.exp(-a * mt - b * mt**2) + mt / p0)**n

# Proper wrapper for ROOT.TF1-compatible PDF
def f_pi0Dalitz(x, p):
    return KrollWada(x[0], M_PI0, me) * EMTFormFactor(x[0], 5.5)

# Setup
f_pi0Dalitz1 = ROOT.TF1("f_pi0Dalitz1", f_pi0Dalitz, 2*me, M_PI0, 0)

In [4]:
def genereate_dalitz(seed):
    rnd = ROOT.TRandom3(seed)
    hists = [ROOT.TH1D(f"hMee{i}", "", 100, 0, M_PI0 * 1.05) for i in range(6)]
    fMee = ROOT.TH1D("fMee", "", 100, 0, M_PI0 * 1.05)
    dphi_hist = ROOT.TH2D("dphi_hist", "dphi_hist", 100, -PI/2, PI/2, 100, 0, 10)
    
    for i in range(Ne):
        if i % 1000_000 == 0:
            print(f"Event {i} in thread {seed}")
    
        # Sample pi0 pT from Hagedorn
        pt = 0
        while True:
            pt = rnd.Uniform(0.01, 15)
            weight = HagedornYield(pt, M_PI0, 377., 0.356, 0.068, 0.7, -8.25)
            if not np.isnan(weight) and not np.isinf(weight):
                break
        if pt == 0:
            continue

        eta = rnd.Uniform(-0.5, 0.5)
        phi = rnd.Uniform(0, 2 * PI)
        pizero = ROOT.TLorentzVector()
        pizero.SetPtEtaPhiM(pt, eta, phi, M_PI0)
        boost_pi0 = pizero.BoostVector()
    
        # Dalitz decay virtual photon
        if True:
            mll = -1
            while mll <= 2 * me or mll >= M_PI0:
                mll = f_pi0Dalitz1.GetRandom()
    
            costh = rnd.Uniform(-1, 1)
            sinth = np.sqrt(1 - costh**2)
            aphi = rnd.Uniform(0, 2 * PI)
    
            E1 = (M_PI0**2 + mll**2) / (2 * M_PI0)
            p1 = np.sqrt(E1**2 - mll**2)
    
            # gamma* and gamma back to back
            gamma_star = ROOT.TLorentzVector(
                p1 * np.cos(aphi) * sinth,
                p1 * np.sin(aphi) * sinth,
                p1 * costh,
                E1
            )
            photon = ROOT.TLorentzVector(
                -gamma_star.Px(), -gamma_star.Py(), -gamma_star.Pz(), M_PI0 - E1
            )
    
            gamma_star.Boost(boost_pi0)
            photon.Boost(boost_pi0)
    
            # e+ e- decay in gamma* frame
            boost_gamma_star = gamma_star.BoostVector()
            Ee = mll / 2.
            pe = np.sqrt(Ee**2 - me**2)
    
            costh2 = rnd.Uniform(-1, 1)
            sinth2 = np.sqrt(1 - costh2**2)
            aphi2 = rnd.Uniform(0, 2 * PI)
    
            eplus = ROOT.TLorentzVector(
                pe * np.cos(aphi2) * sinth2,
                pe * np.sin(aphi2) * sinth2,
                pe * costh2,
                Ee
            )
            eminus = ROOT.TLorentzVector(
                -eplus.Px(), -eplus.Py(), -eplus.Pz(), Ee
            )
    
            eplus.Boost(boost_gamma_star)
            eminus.Boost(boost_gamma_star)
    
            ptSum = (eplus + eminus).Pt()
            #if eplus.Pt() < 0.1 or eminus.Pt() < 0.1:
            #    continue
            delta_phi = eplus.Phi() - eminus.Phi()
            if delta_phi < -PI:
                delta_phi += 2 * PI
            elif delta_phi > PI:
                delta_phi -= 2 * PI
            #ptSum = pt
            #break  # accept all events, can add cuts later
    
            fMee.Fill(mll, weight)
            hists[0].Fill(mll, weight)
            if 0 < ptSum < 0.1: hists[1].Fill(mll, weight)
            elif 0.1 <= ptSum < 0.25: hists[2].Fill(mll, weight)
            elif 0.25 <= ptSum < 0.5: hists[3].Fill(mll, weight)
            elif 0.5 <= ptSum < 1.0: hists[4].Fill(mll, weight)
            elif 1.0 <= ptSum < 2.0: hists[5].Fill(mll, weight)
            dphi_hist.Fill(delta_phi, ptSum, weight)
            if ptSum > 2 and abs(delta_phi) > 2*PI/3:
                print(f"Warning: ptSum of {ptSum:.1f} = {eplus.Pt():.1f}+{eminus.Pt():.1f}, delta_phi of {delta_phi:.2f} = {eplus.Phi():.2f}+{eminus.Phi():.2f} in event {i} in thread {seed}")

    return [hists, fMee, dphi_hist]


In [5]:
Ntr = N_tr
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(genereate_dalitz, [i for i in range(Ntr)])
pool.close()

Event 0 in thread 4Event 0 in thread 9Event 0 in thread 5Event 0 in thread 2Event 0 in thread 6Event 0 in thread 7





Event 0 in thread 0Event 0 in thread 1

Event 0 in thread 8
Event 0 in thread 3


In [6]:
hists = output_array[0][0]
fMee = output_array[0][1]
dphi_hist = output_array[0][2]
for ithread in range(1, Ntr):
    for i, h in enumerate(hists):
        hists[i].Add(output_array[ithread][0][i])
    fMee.Add(output_array[ithread][1])
    dphi_hist.Add(output_array[ithread][2])

In [9]:
# Normalize
for h in hists:
    norm = h.Integral(1, 6)
    if norm > 0:
        h.Scale(1.0 / norm)
fMee.Scale(1.0 / fMee.Integral(1, 6))

# Plot
c1 = ROOT.TCanvas("c1", "", 1200, 1200)
c1.SetLogy()
fMee.SetMarkerStyle(20)
fMee.SetMarkerColor(ROOT.kGray)
fMee.SetMinimum(1e-4)
fMee.SetMaximum(1)
fMee.GetXaxis().SetTitle("M_{ee} [GeV/c^{2}]")
fMee.GetYaxis().SetTitle("yield")
fMee.Draw("p")

for i, h in enumerate(hists):
    h.SetLineColor(colors[i])
    h.SetLineWidth(2)
    h.Draw("same l")

c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [8]:
#plotting dphi_hist
c2 = ROOT.TCanvas("c2", "", 1600, 1000)
c2.Divide(3,2)
pt_bins = [0, 2, 4, 6, 9, 20, 100]
legends= []
for ipt in range (6):
    c2.cd(ipt + 1)
    dphi_hist_proj = dphi_hist.ProjectionX(f"dphi_hist_proj_{ipt}", pt_bins[ipt]+1, pt_bins[ipt + 1])
    dphi_hist_proj.SetMarkerStyle(20)
    dphi_hist_proj.SetMarkerColor(ROOT.kRed)
    dphi_hist_proj.GetXaxis().SetTitle("#Delta#phi [rad]")
    dphi_hist_proj.GetYaxis().SetTitle("yield")
    ROOT.gPad.SetLogy()
    legends.append(ROOT.TLegend(0.1, 0.1, 0.8, 0.3))
    legends[-1].AddEntry(dphi_hist_proj, f"{pt_bins[ipt]/10:.1f} < pT < {pt_bins[ipt + 1]/10:.1f} GeV/c", "p")
    legends[-1].SetFillStyle(0)
    legends[-1].SetLineWidth(0)
    dphi_hist_proj.Draw("p")
    legends[-1].Draw()
c2.Draw()